In [1]:
import numpy as np
import pandas as pd

# Make printing easier
np.set_printoptions(precision=6, suppress=True)


In [2]:
def q_value(x, G, c):
    """Quadratic objective: 0.5 x^T G x + c^T x."""
    x = np.asarray(x, dtype=float)
    return 0.5 * x @ (G @ x) + c @ x


def q_grad(x, G, c):
    """Gradient of quadratic objective."""
    x = np.asarray(x, dtype=float)
    return G @ x + c


def is_feasible(x, A, b, tol=1e-8):
    """Check A x >= b componentwise."""
    return np.all(A @ x >= b - tol)


In [4]:
def primal_active_set_qp(G, c, A, b, x0, W0=None, max_iter=50, tol=1e-8, verbose=True):
    """Primal active-set method for strictly convex QP:
        minimize 0.5 x^T G x + c^T x
        subject to A x >= b.
    G must be symmetric positive definite.
    """
    G = np.asarray(G, dtype=float)
    c = np.asarray(c, dtype=float)
    A = np.asarray(A, dtype=float)
    b = np.asarray(b, dtype=float)
    x = np.asarray(x0, dtype=float)

    m, n = A.shape
    if W0 is None:
        W = []
    else:
        W = list(W0)

    history = []
    k = 0

    if not is_feasible(x, A, b, tol):
        raise ValueError("Initial point x0 is not feasible.")

    if verbose:
        print("Start primal active-set QP")
        print(f"x^0 = {x}, W_0 = {W}\n")

    while k < max_iter:
        g = q_grad(x, G, c)

        # Subproblem: minimize 0.5 p^T G p + g^T p, s.t. A_W p = 0
        if len(W) == 0:
            # Unconstrained direction
            p = -np.linalg.solve(G, g)
        else:
            A_W = A[W, :]           # |W| x n
            # KKT system:
            # [ G   A_W^T ] [ p ] = -g
            # [ A_W  0   ] [ u ] =  0
            KKT = np.block([[G, A_W.T],
                            [A_W, np.zeros((len(W), len(W)))]])
            rhs = np.concatenate([-g, np.zeros(len(W))])
            sol = np.linalg.solve(KKT, rhs)
            p = sol[:n]

        # Record before deciding step type
        rec = {
            "k": k,
            "x": x.copy(),
            "W": W.copy(),
            "g": g.copy(),
            "p": p.copy(),
        }

        if np.linalg.norm(p) <= tol:
            # Multiplier check branch
            if len(W) == 0:
                # Unconstrained optimum
                lambdas = np.array([])
                rec["step_type"] = "p=0, W empty -> optimal"
                history.append(rec)
                if verbose:
                    print(f"Iter {k}: p = 0, W empty, stop.")
                break

            A_W = A[W, :]
            # Stationarity: G x + c - A_W^T lambda = 0
            lambdas = np.linalg.solve(A_W @ A_W.T, A_W @ (G @ x + c))
            rec["lambdas"] = lambdas.copy()

            if verbose:
                print(f"Iter {k}: p = 0, multipliers on W = {lambdas}")

            if np.all(lambdas >= -tol):
                rec["step_type"] = "p=0, all lambda>=0 -> optimal"
                history.append(rec)
                if verbose:
                    print("All multipliers nonnegative, KKT satisfied, stop.")
                break
            else:
                # Remove one most negative multiplier
                idx = int(np.argmin(lambdas))
                j = W[idx]
                rec["removed"] = j
                rec["step_type"] = f"p=0, lambda_{j}<0 -> remove {j} from W"
                history.append(rec)
                if verbose:
                    print(f"Remove constraint {j} from W (lambda={lambdas[idx]:.4g}).")
                W.pop(idx)
                # x unchanged
                k += 1
                continue
        else:
            # Descent direction branch
            # Compute blocking step length
            alphas = [1.0]
            blocking_indices = []

            for i in range(m):
                if i in W:
                    continue
                a_i = A[i, :]
                aTp = a_i @ p
                if aTp < -tol:
                    alpha_i = (b[i] - a_i @ x) / aTp
                    if alpha_i >= 0:
                        alphas.append(alpha_i)
                        blocking_indices.append(i)

            alpha = min(alphas)
            rec["alpha"] = alpha

            if verbose:
                print(f"Iter {k}: x = {x}, W = {W}")
                print(f"         p = {p}, alpha = {alpha:.4g}")

            x_new = x + alpha * p

            if alpha < 1 - 1e-12:
                # Some constraint becomes active (blocking)
                # Pick the constraint that attains alpha
                chosen = None
                for i in blocking_indices:
                    a_i = A[i, :]
                    aTp = a_i @ p
                    if aTp < -tol:
                        alpha_i = (b[i] - a_i @ x) / aTp
                        if abs(alpha_i - alpha) <= 1e-8:
                            chosen = i
                            break
                if chosen is None and len(blocking_indices) > 0:
                    chosen = blocking_indices[0]

                rec["added"] = chosen
                rec["step_type"] = f"descent, alpha<1, add {chosen} to W"
                history.append(rec)

                x = x_new
                if chosen is not None and chosen not in W:
                    W.append(chosen)
                k += 1
                continue
            else:
                # Full step, no new blocking constraint
                rec["step_type"] = "descent, alpha=1, W unchanged"
                history.append(rec)
                x = x_new
                k += 1
                continue

    history_df = []
    for h in history:
        row = {
            "k": h["k"],
            "x": h["x"],
            "W": h["W"],
            "step_type": h.get("step_type", ""),
            "alpha": h.get("alpha", np.nan),
        }
        history_df.append(row)
    history_df = pd.DataFrame(history_df)

    return x, history, history_df


In [ ]:
# Example: Problem 23 from the homework (max f -> min q)
# q(x) = x1^2 + x2^2 - 6 x1 - 4 x2 + 13
G = 2 * np.eye(2)
c = np.array([-6.0, -4.0])

# Constraints: A x >= b
# 1) x1 + x2 <= 3  ->  -x1 - x2 >= -3
# 2) x1 >= 0       ->   x1 >= 0
# 3) x2 >= 0       ->   x2 >= 0
A = np.array([
    [-1.0, -1.0],
    [ 1.0,  0.0],
    [ 0.0,  1.0],
])
b = np.array([-3.0, 0.0, 0.0])

x0 = np.array([1/3, 1/3]) # Feasible starting point
W0 = []

x_star, history, history_df = primal_active_set_qp(G, c, A, b, x0, W0, verbose=True)

print("\nOptimal x (for min q) =", x_star)
print("Objective q(x*) =", q_value(x_star, G, c))
print("Original max f(x*) =", -q_value(x_star, G, c))


Start primal active-set QP
x^0 = [0.333333 0.333333], W_0 = []

Iter 0: x = [0.333333 0.333333], W = []
         p = [2.666667 1.666667], alpha = 0.5385
Iter 1: x = [1.769231 1.230769], W = [0]
         p = [ 0.230769 -0.230769], alpha = 1
Iter 2: p = 0, multipliers on W = [2.]
All multipliers nonnegative, KKT satisfied, stop.

Optimal x (for min q) = [2. 1.]
Objective q(x*) = -11.0
Original max f(x*) = 11.0


In [5]:
history_df

,k,x,W,step_type,alpha
0,0,"[0.0, 0.0]",[],"descent, alpha<1, add 0 to W",0.6
1,1,"[1.7999999999999998, 1.2]",[0],"descent, alpha=1, W unchanged",1.0
2,2,"[2.0, 1.0]",[0],"p=0, all lambda>=0 -> optimal",NaN
